In [4]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [5]:
from fastai.text import *

In [6]:
data_path = Config.data_path()/'amazon-fine-foods'; data_path.mkdir(parents=True, exist_ok=True)

In [7]:
!cat {data_path}/'sample.txt'

product/productId: B001E4KFG0
review/userId: A3SGXH7AUHU8GW
review/profileName: delmartian
review/helpfulness: 1/1
review/score: 5.0
review/time: 1303862400
review/summary: Good Quality Dog Food
review/text: I have bought several of the Vitality canned dog food products and have found them all to be of good quality. The product looks more like a stew than a processed meat and it smells better. My Labrador is finicky and she appreciates this product better than  most.

product/productId: B00813GRG4
review/userId: A1D87F6ZCVE5NK
review/profileName: dll pa
review/helpfulness: 0/0
review/score: 1.0
review/time: 1346976000
review/summary: Not as Advertised
review/text: Product arrived labeled as Jumbo Salted Peanuts...the peanuts were actually small sized unsalted. Not sure if this was an error or if the vendor intended to represent the product as "Jumbo".

product/productId: B000LQOCH0
review/userId: ABXLMWJIXXAIN
review/profileName: Natalia Corres "Natalia Corres"
review/helpfulness: 1/1


In [25]:
txt_file = data_path/'sample.txt'
txt_file = data_path/'finefoods.txt'

In [26]:
n_reviews = !grep -o 'product/productId:' {txt_file} | wc -l
n_reviews = int(n_reviews[0]); n_reviews

568454

Check that the data is correct.

In [126]:
feature_columns = ['product/productId', 'review/userId', 'review/profileName',
                   'review/helpfulness', 'review/score', 'review/time', 'review/summary', 'review/text']

In [127]:
df = pd.DataFrame(index=range(n_reviews), columns=feature_columns)

In [129]:
with open(txt_file, 'r', encoding='latin-1') as f:
    # The last element would be empty since there is a trailing double newline at the end of file
    reviews = f.read().split('\n\n')[:-1]

In [130]:
reviews = [r.split('\n') for r in reviews]

Certain reviews contained an extra line just after the profile name. This messed up creating the dataframe so we delete them.

In [131]:
for l in reviews[94197]: print(l)

product/productId: B002RIZUQ2
review/userId: AS2DLXUWDK0GP
review/profileName: MABEL "Tell us about yourself!
88 years old. ...
review/helpfulness: 1/1
review/score: 4.0
review/time: 1289088000
review/summary: delicious
review/text: if you prefer a tasteful and less strong flavored coffee, I reommend Green Mountain's hazelnut.  I have not been a coffee drinker, but upon tasting this brew, I drink it daily.


In [132]:
for i in range(len(reviews)):
    if len(reviews[i]) != len(feature_columns):
        del reviews[i][3]

In [138]:
#regex.match('product/productId: B001E4KFG0').groups()

In [141]:
feature_columns

['product/productId',
 'review/userId',
 'review/profileName',
 'review/helpfulness',
 'review/score',
 'review/time',
 'review/summary',
 'review/text']

In [156]:
%%time
regex = re.compile(r"^([^:]*): (.*)$") # omits the category suffix such as 'product/productId: '
for i, review in enumerate(reviews):
    for line in review:
        column, data = regex.match(line).groups()
        assert (column in feature_columns)
        df.iloc[i][column] = data
    if i % 100000 == 0: print(i)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000
440000
450000
460000
470000
480000
490000
500000
510000
520000
530000
540000
550000
560000
CPU times: user 6min 8s, sys: 580 ms, total: 6min 9s
Wall time: 6min 9s


In [ ]:
df.to_csv(data_path/'finefoods.csv', index=False)

In [245]:
df = pd.read_csv(data_path/'finefoods.csv')

In [246]:
rows_with_nans = df[df.isnull().any(axis=1)]; rows_with_nans.head()

,product/productId,review/userId,review/profileName,review/helpfulness,review/score,review/time,review/summary,review/text
25509,B000LKZB4Y,A36BVYD0NT7Z0F,NaN,0/0,5.0,1314576000,These are the best mints and no aspartame or BHT,I was so shocked to find out that almost all g...
33958,B00412W76S,A3TJPSWY2HE4BS,"S. Layton ""homeschool blogger""",1/24,2.0,1173312000,NaN,I only used two maybe three tea bags and got p...
38874,B000AYDGZ2,A36BVYD0NT7Z0F,NaN,2/3,1.0,1278374400,doesn't anyone care that they are putting BHT ...,I called Kellogg's to see why Special K red be...
40548,B00020HHRW,A3TJPSWY2HE4BS,"S. Layton ""homeschool blogger""",1/24,2.0,1173312000,NaN,I only used two maybe three tea bags and got p...
49800,B000CRHQN0,A2LYFY32LXQDON,NaN,0/0,2.0,1282608000,They were melted and the chocolate had turned ...,We love these bars but i won't order them ship...


In [247]:
print(df.shape)
print(rows_with_nans.shape)

(568454, 8)
(43, 8)


In [248]:
df = df.drop(list(rows_with_nans.index.values));

In [249]:
df.shape

(568411, 8)

In [250]:
separator = ' '
df["review/summary_and_text"] = df["review/summary"].map(str) + separator + df["review/text"]

In [251]:
df.head()

,product/productId,review/userId,review/profileName,review/helpfulness,review/score,review/time,review/summary,review/text,review/summary_and_text
0,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1/1,5.0,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...,Good Quality Dog Food I have bought several of...
1,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0/0,1.0,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,Not as Advertised Product arrived labeled as J...
2,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1/1,4.0,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...,"""Delight"" says it all This is a confection tha..."
3,B000UA0QIQ,A395BORC6FGVXV,Karl,3/3,2.0,1307923200,Cough Medicine,If you are looking for the secret ingredient i...,Cough Medicine If you are looking for the secr...
4,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0/0,5.0,1350777600,Great taffy,Great taffy at a great price. There was a wid...,Great taffy Great taffy at a great price. The...
